In [1]:

import numpy as np
import scipy as sp
import h5py
import os

import SimData as sim_data
import Simulator as sim_system
import Optimization as opt

In [2]:


##! keys values are the columns of interest of exp data,
##! but the values are the ones used by the simulator 
##* the experimental data is read as it is presented, modfications are added later in the Simaulator object
##* gas species must be consistent with the chemical equations from the json file


schema = {
    "Pressure":     "pressure",
    "CurrentName":  "current",
    "WallTemp":     "Tw",
    "T near wall":  "Tnw", 
    "Tgas":         "Tgas",
    "O mean":       "O",
    "N":            "N",
    "CO mean":      "CO",
    "gamma.1":      "gamma_exp"
}


###* Input files and function

reactions_file = "reactionsSimpleV2.json"

const_dict = {
        "F0": 1.5e15,           # cm^-2
        "S0": 3e13,             # cm^-2
        
        "R": 0.00831442,        # kJ/mol*K
        "kBoltz": 1.380649e-23, # J/K
}

initial_state_dict = {'O_F': 0.1, 'O2_F':0.1 ,'O_S': 0.1, 'Vdb_S':0.1, 
                      'Odb_S': 0.1, 'CO_F': 0.1, 'CO2_F':0.1, 'CO_S': 0.1, 
                      'COdb_S': 0.1}

###* Functions for the data transformation
def compute_flux(const_dict, exp_dict, specie, molar_mass):
    den = exp_dict.get(specie, 0.0)
    v_th = np.sqrt((8.0 * const_dict['R'] * 1000 * exp_dict['Tnw'])/(molar_mass * np.pi))
    flux = 0.25 * v_th * den * 100
    return flux


def compute_remaining_flux(const_dict, exp_dict, molar_mass): 
    den = exp_dict['N'] - exp_dict['O'] - exp_dict['CO']
    v_th = np.sqrt((8.0 * const_dict['R'] * 1000 * exp_dict['Tnw'])/(molar_mass * np.pi))
    flux = 0.25 * v_th * den * 100
    return flux

####? EavgMB data extracted from the Booth et al. 2019 paper
p_data_exp = [0.2, 0.3, 0.4, 0.5, 0.6, 0.75, 1.5]
EavgMB_data = [1.04, 0.91, 0.87, 0.83, 0.77, 0.5, 0.001]
interpolator = sp.interpolate.interp1d(p_data_exp, EavgMB_data, kind='linear', fill_value=0.001, bounds_error=False)


transformations_exp = {
    'Tw':       lambda const_dict, exp_dict: exp_dict['Tw'] + 273.15,
    'fluxO2' :  lambda const_dict, exp_dict: compute_remaining_flux(const_dict, exp_dict, 0.032),
    'fluxO' :   lambda const_dict, exp_dict: compute_flux(const_dict, exp_dict, 'O', 0.016),
    'fluxCO':   lambda const_dict, exp_dict: compute_flux(const_dict, exp_dict, 'CO', 0.028),
    'fluxCO2':  lambda const_dict, exp_dict: compute_flux(const_dict, exp_dict, 'CO2', 0.048),
    'EavgMB':   lambda const_dict, exp_dict: interpolator(exp_dict['pressure']).item(),
    'Ion':      lambda const_dict, exp_dict: 1e14 * exp_dict["current"]
}

input_folder_path = "Experimental_data_Paper"

output_folder_path = "Buffer_Data"
exp_data_file = input_folder_path + ".hdf5"

In [3]:

data_loader = sim_data.DataLoader(schema, folder_path=input_folder_path, output_file=exp_data_file, output_folder=output_folder_path)
data_loader.load_data(force_update=True)


In [4]:

exp_file = os.path.join(output_folder_path, exp_data_file)

sim = sim_system.Simulator(reactions_file, const_dict, exp_file, initial_state_dict, transformations_exp=transformations_exp)

# print()
# print(sim.output_parser['rates_model'])
# print(sim.output_parser['reactions_list'])


  d[CO2_F]/dt = -CO2_F*r_29 + r_28*(-CO2_F - CO_F - O2_F - O_F + 1.0)
  d[CO_F]/dt = -CO_F*O_F*r_34 - 0.02*CO_F*O_S*r_39 - 0.02*CO_F*Odb_S*r_60 - 0.02*CO_F*Vdb_S*r_59 - CO_F*r_31 - CO_F*r_33 - 0.02*CO_F*r_35*(-CO_S - COdb_S - O_S - Odb_S - Vdb_S + 1.0) + r_30*(-CO2_F - CO_F - O2_F - O_F + 1.0)
  d[CO_S]/dt = CO_F*r_35*(-CO_S - COdb_S - O_S - Odb_S - Vdb_S + 1.0) - CO_S*O_F*r_38 - CO_S*r_36 - CO_S*r_42 - CO_S*r_43 - CO_S*r_44 - CO_S*r_45 + r_32*(-CO_S - COdb_S - O_S - Odb_S - Vdb_S + 1.0)
  d[COdb_S]/dt = CO_F*Vdb_S*r_59 - COdb_S*O_F*r_61 - COdb_S*r_53 - COdb_S*r_54 - COdb_S*r_55 - COdb_S*r_56 - COdb_S*r_58 + Vdb_S*r_48
  d[O2_F]/dt = -O2_F*O_F*r_15 - O2_F*r_10 - O2_F*r_12 - O2_F*r_14 + r_9*(-CO2_F - CO_F - O2_F - O_F + 1.0)
  d[O_F]/dt = -CO_F*O_F*r_34 - 0.02*CO_S*O_F*r_38 - 0.02*COdb_S*O_F*r_61 - O2_F*O_F*r_15 - 2*O_F**2*r_8 - 0.02*O_F*O_S*r_7 - 0.02*O_F*Odb_S*r_27 - 0.02*O_F*Vdb_S*r_26 - O_F*r_11 - O_F*r_2 - O_F*r_4 - 0.02*O_F*r_5*(-CO_S - COdb_S - O_S - Odb_S - Vdb_S + 1.0) + r_1*(-

In [5]:


def func_optimization(params):
    
    E, nu_D = params

    dict_mod_vec = [
    {"id": 1, "rate": "adsorption", "model_dict": {"E": E}},
    {"id": 8, "rate": None, "model_dict": {"nu_D": nu_D}}
    ]
    
    return dict_mod_vec

loss_function = lambda exp, teo: np.mean((np.reciprocal(exp)*(exp-teo))**2)


# optimzer = opt.Optimizer(sim, func_optimization, loss_function)

In [6]:

if __name__ == '__main__':
    exp_file = os.path.join(output_folder_path, exp_data_file)

    sim = sim_system.Simulator(reactions_file, const_dict, exp_file, initial_state_dict, transformations_exp=transformations_exp)
    optimizer = opt.Optimizer(sim, func_optimization, loss_function)

    global_bounds = [(1.0, 3.0), (5e12, 10e13)]

    config = {
        "bounds": global_bounds,
        "nb_calls": 3, # 5
        "de_maxiter": 10,
        "top_k": 1,
        "local_attempts": 2,
        "epsilon_local": 1e-2
    }


    # loss = optimizer.objective_function(np.array([0.0, 1e13]))
    
    # print(loss)
    
    best_local, best_local_loss = optimizer.hybrid_optimization_search(config)


  d[CO2_F]/dt = -CO2_F*r_29 + r_28*(-CO2_F - CO_F - O2_F - O_F + 1.0)
  d[CO_F]/dt = -CO_F*O_F*r_34 - 0.02*CO_F*O_S*r_39 - 0.02*CO_F*Odb_S*r_60 - 0.02*CO_F*Vdb_S*r_59 - CO_F*r_31 - CO_F*r_33 - 0.02*CO_F*r_35*(-CO_S - COdb_S - O_S - Odb_S - Vdb_S + 1.0) + r_30*(-CO2_F - CO_F - O2_F - O_F + 1.0)
  d[CO_S]/dt = CO_F*r_35*(-CO_S - COdb_S - O_S - Odb_S - Vdb_S + 1.0) - CO_S*O_F*r_38 - CO_S*r_36 - CO_S*r_42 - CO_S*r_43 - CO_S*r_44 - CO_S*r_45 + r_32*(-CO_S - COdb_S - O_S - Odb_S - Vdb_S + 1.0)
  d[COdb_S]/dt = CO_F*Vdb_S*r_59 - COdb_S*O_F*r_61 - COdb_S*r_53 - COdb_S*r_54 - COdb_S*r_55 - COdb_S*r_56 - COdb_S*r_58 + Vdb_S*r_48
  d[O2_F]/dt = -O2_F*O_F*r_15 - O2_F*r_10 - O2_F*r_12 - O2_F*r_14 + r_9*(-CO2_F - CO_F - O2_F - O_F + 1.0)
  d[O_F]/dt = -CO_F*O_F*r_34 - 0.02*CO_S*O_F*r_38 - 0.02*COdb_S*O_F*r_61 - O2_F*O_F*r_15 - 2*O_F**2*r_8 - 0.02*O_F*O_S*r_7 - 0.02*O_F*Odb_S*r_27 - 0.02*O_F*Vdb_S*r_26 - O_F*r_11 - O_F*r_2 - O_F*r_4 - 0.02*O_F*r_5*(-CO_S - COdb_S - O_S - Odb_S - Vdb_S + 1.0) + r_1*(-

AttributeError: 'Optimizer' object has no attribute 'functional_calls'

In [ ]:


##### add the Error Propagation Class: error in model parameters and experimental data as well
### add the CO reactions